If Dockerfiles have not been modified, connect to the Jupyter server with:  
- ```http://localhost:8008/tree?token=encode-point-clouds-cpu``` to run Torch on CPU  
- ```http://localhost:8016/tree?token=encode-point-clouds-cuda121``` to run Torch on CUDA 12.1

This notebook describes a pipeline to encode point clouds derived from [Li et al (2021)](https://github.com/feiran-l/rotation-invariant-pointcloud-analysis), and uses their pretrained model trained on ModelNet40.  
It takes a ```.csv``` without headers describing 1024 cartesian (xyz) coordinates of a point cloud and returns a 512×1 matrix as a ```.json``` encoding the point cloud.  

> F. Li, K. Fujiwara, F. Okura and Y. Matsushita, "A Closer Look at Rotation-invariant Deep Point Cloud Analysis," 2021 IEEE/CVF International Conference on Computer Vision (ICCV), Montreal, QC, Canada, 2021, pp. 16198-16207, doi: 10.1109/ICCV48922.2021.01591.

In [1]:
target_dir = "data"
input_dir = "place-pulse-singapore-segmented-point-clouds-split-sampled"
output_dir = "place-pulse-singapore-segmented-point-clouds-split-encoded"

In [2]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.nn.init as init

from contextlib import redirect_stdout
import csv
import io
import itertools
import json
import os
from pathlib import Path
import sys

In [3]:
class TransformNet(nn.Module):
    def __init__(self):
        super(TransformNet, self).__init__()
        self.k = 3
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False), self.bn1, nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=1, bias=False), self.bn2, nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv1d(128, 1024, kernel_size=1, bias=False), self.bn3, nn.LeakyReLU(negative_slope=0.2))
        self.linear1 = nn.Linear(1024, 512, bias=False)
        self.bn3 = nn.BatchNorm1d(512)
        self.linear2 = nn.Linear(512, 256, bias=False)
        self.bn4 = nn.BatchNorm1d(256)
        self.transform = nn.Linear(256, 3 * 3)
        init.constant_(self.transform.weight, 0)
        init.eye_(self.transform.bias.view(3, 3))

    def forward(self, x):
        bs = x.size(0)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.max(dim=-1, keepdim=False)[0]
        x = self.conv3(x)
        x = x.max(dim=-1, keepdim=False)[0]
        x = F.leaky_relu(self.bn3(self.linear1(x)), negative_slope=0.2)
        x = F.leaky_relu(self.bn4(self.linear2(x)), negative_slope=0.2)
        x = self.transform(x).view(bs, 3, 3)
        return x


def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    return idx


def get_graph_feature(x, k=20, idx=None):
    batch_size = x.size(0)
    num_points = x.size(2)
    x = x.view(batch_size, -1, num_points)
    if idx is None:
        idx = knn(x, k=k)   # (batch_size, num_points, k)
    idx_base = torch.arange(0, batch_size, device=x.device).view(-1, 1, 1)*num_points
    idx = idx + idx_base
    idx = idx.view(-1)
    _, num_dims, _ = x.size()
    x = x.transpose(2, 1).contiguous()   # (bs, num_points, num_dims) -> (bs*num_points, num_dims) #   bs * num_points * k + range(0, bs*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2).contiguous()
    return feature


class DGCNN(nn.Module):
    def __init__(self, k=25, output_channels=40):
        super(DGCNN, self).__init__()
        self.k = k
        self.transform_net = TransformNet()
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(1024)

        # selector module
        self.linearsel1 = nn.Conv1d(24, 512, kernel_size=1)
        self.bnsel1 = nn.BatchNorm1d(512)
        self.linearsel2 = nn.Conv1d(512,256, kernel_size=1)
        self.bnsel2 = nn.BatchNorm1d(256)
        self.linearsel3 = nn.Linear(256, 24)

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, kernel_size=1, bias=False), self.bn1, nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False), self.bn2, nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 128, kernel_size=1, bias=False), self.bn3, nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(128*2, 256, kernel_size=1, bias=False), self.bn4, nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, 1024, kernel_size=1, bias=False), self.bn5, nn.LeakyReLU(negative_slope=0.2))
        
        self.linear1 = nn.Linear(1024*2, 512, bias=False)
        self.bn6 = nn.BatchNorm1d(512)
        self.dp1 = nn.Dropout()
        self.linear2 = nn.Linear(512, 512)
        self.bn7 = nn.BatchNorm1d(512)
        self.dp2 = nn.Dropout()
        self.linear3 = nn.Linear(512, output_channels)


    def forward(self, x):
        bs = x.size(0)

        # pose selector
        xf = x.permute(0, 2, 3, 1).view(bs, 1024*3, 24)
        s = F.leaky_relu(self.bnsel1(self.linearsel1(xf.transpose(2, 1))), negative_slope=0.2)
        s = F.leaky_relu(self.bnsel2(self.linearsel2(s)), negative_slope=0.2)
        s = F.adaptive_max_pool1d(s, 1).view(bs, -1)
        s = F.softmax(self.linearsel3(s), dim=1).unsqueeze(-1)
        x = torch.bmm(xf, s).view(bs, 1024, 3).permute(0 ,2, 1) # weighting the 24 poses

        # DGCNN
        x0 = get_graph_feature(x, k=self.k)  # (bs, 3, n_points) -> (bs, 3*2, n_points, k)
        t = self.transform_net(x0)  # (bs, 3, 3)
        x = x.transpose(2, 1)  # (bs, 3, n_points) -> (bs, n_points, 3)
        x = torch.bmm(x, t)  # (bs, n_points, 3) * (bs, 3, 3) -> (bs, n_points, 3)
        x = x.transpose(2, 1)
        x = get_graph_feature(x, k=self.k)
        x = self.conv1(x)
        x1 = x.max(dim=-1, keepdim=False)[0]
        x = get_graph_feature(x1, k=self.k)
        x = self.conv2(x)
        x2 = x.max(dim=-1, keepdim=False)[0]
        x = get_graph_feature(x2, k=self.k)
        x = self.conv3(x)
        x3 = x.max(dim=-1, keepdim=False)[0]
        x = get_graph_feature(x3, k=self.k)
        x = self.conv4(x)
        x4 = x.max(dim=-1, keepdim=False)[0]
        x = torch.cat((x1, x2, x3, x4), dim=1)
        x = self.conv5(x)
        
        x1 = F.adaptive_max_pool1d(x, 1).view(bs, -1)
        x2 = F.adaptive_avg_pool1d(x, 1).view(bs, -1)
        x = torch.cat((x1, x2), 1)
                
        x = F.leaky_relu(self.bn6(self.linear1(x)), negative_slope=0.2)
        x = self.dp1(x)
        # x = F.leaky_relu(self.bn7(self.linear2(x)), negative_slope=0.2)
        x = self.linear2(x)
        # x = self.dp2(x)
        # x = self.linear3(x)
        return x

In [ ]:
# Model Hyperparameters
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

torch.manual_seed(123)
torch.cuda.manual_seed(123)
model = DGCNN().to(device)
model = nn.DataParallel(model, device_ids=[device])
model.load_state_dict(torch.load(os.path.join(os.getcwd(), "modelnet40_checkpoint.t7"), device))
model.eval()
vcand = np.loadtxt(os.path.join(os.getcwd(), "all_id.txt")).astype(int) # view candidate
print('init done')

cuda


In [17]:
def feature_extractor(pcd):
    pcd = generate_24_poses(pcd)
    pcd = np.stack(pcd)
    pcd = torch.from_numpy(pcd).unsqueeze(0).to(device).float()
    with torch.no_grad():
        ids = torch.from_numpy(vcand).to(device).long()
        tmp_pred = []
        for vw in range(24):
            print(ids[vw])
            tmp_pred.append(model(pcd[:, ids[vw]]).detach().cpu().numpy())
        pred = np.sum(np.concatenate(tmp_pred), axis=0)
        return pred
    
# From RIPC
def generate_24_poses(pcd):
    """
    @param pcd: N*3 point cloud
    @return: a list of the 24 poses
    """
    ## get principle axes
    _, eig_vecs = np.linalg.eigh(pcd.T @ pcd)
    if np.linalg.det(eig_vecs) < 0:
        eig_vecs[:, 2] = -1.0 * eig_vecs[:, 2]
    e1, e2, e3 = eig_vecs[:, 0], eig_vecs[:, 1], eig_vecs[:, 2]

    ## get all 48 possible combos, among them, 
    ## 24 are with det 1 (rotations), and the other 24 are with det -1 (improper rotations)
    all_R = list(itertools.permutations([e1, e2, e3])) + list(itertools.permutations([-e1, -e2, -e3])) + \
             list(itertools.permutations([-e1, e2, e3])) + list(itertools.permutations([e1, -e2, e3])) + \
             list(itertools.permutations([e1, e2, -e3])) + list(itertools.permutations([-e1, -e2, e3])) + \
             list(itertools.permutations([-e1, e2, -e3])) + list(itertools.permutations([e1, -e2, -e3]))
    all_R = [np.array(x) for x in all_R]

    ## remove improper rotations, the left are the 24 ambiguities
    all_R = [x for x in all_R if np.linalg.det(x) > 0]

    res = [pcd @ R for R in all_R]
    return res

In [ ]:
point_clouds_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, input_dir)):
    for filename in filenames:
        if Path(os.path.join(dirpath, filename)).is_file():
            point_clouds_path.append(os.path.join(dirpath, filename).split(os.path.join(target_dir, input_dir) + '/')[-1])

for path in point_clouds_path:
    filename_no_ext = path.split('.')[0]
    if Path(os.path.join(target_dir, output_dir, filename_no_ext + ".json")).is_file():
        continue
    if not Path(os.path.dirname(os.path.join(target_dir, output_dir, filename_no_ext + ".json"))).is_dir():
        Path(os.path.dirname(os.path.join(target_dir, output_dir, filename_no_ext + ".json"))).mkdir(parents=True, exist_ok=True)
    point_cloud = []
    with open(os.path.join(target_dir, input_dir, path), 'r') as fp:
        csv_reader = csv.reader(fp)
        for row in csv_reader:
            point_cloud.append(list(map(float, row[:3])))
    if len(point_cloud) <= 0:
        continue
    point_cloud_encoded = feature_extractor(np.array(point_cloud)).tolist()
    with open(os.path.join(target_dir, output_dir, filename_no_ext + ".json"), 'w') as fp:
        json.dump(point_cloud_encoded, fp)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23])
tensor([ 1,  0,  3,  2,  6,  7,  4,  5, 11, 10,  9,  8, 14, 15, 12, 13, 19, 18,
        17, 16, 21, 20, 23, 22])
tensor([ 2,  3,  0,  1,  7,  6,  5,  4,  9,  8, 11, 10, 15, 14, 13, 12, 17, 16,
        19, 18, 22, 23, 20, 21])
tensor([ 3,  2,  1,  0,  5,  4,  7,  6, 10, 11,  8,  9, 13, 12, 15, 14, 18, 19,
        16, 17, 23, 22, 21, 20])
tensor([ 4,  5,  6,  7,  8,  9, 10, 11,  0,  1,  2,  3, 16, 17, 18, 19, 20, 21,
        22, 23, 12, 13, 14, 15])
tensor([ 5,  4,  7,  6, 10, 11,  8,  9,  3,  2,  1,  0, 18, 19, 16, 17, 23, 22,
        21, 20, 13, 12, 15, 14])
tensor([ 6,  7,  4,  5, 11, 10,  9,  8,  1,  0,  3,  2, 19, 18, 17, 16, 21, 20,
        23, 22, 14, 15, 12, 13])
tensor([ 7,  6,  5,  4,  9,  8, 11, 10,  2,  3,  0,  1, 17, 16, 19, 18, 22, 23,
        20, 21, 15, 14, 13, 12])
tensor([ 8,  9, 10, 11,  0,  1,  2,  3,  4,  5,  6,  7, 20, 21, 22, 23, 12, 13,
        14, 15, 